In [1]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential # constructor
from keras.layers import Dense

2024-05-08 10:42:58.732578: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 10:42:58.736431: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 10:42:58.814876: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 10:42:59.897654: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [3]:
# Download the data and read it as pandas dataframe
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
# Check number of datapoints
concrete_data.shape

(1030, 9)

In [5]:
# Check is there is any missing values
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [7]:
# Split data into predictors and target
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']]
target = concrete_data['Strength']

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
# Normalize the data by substracting the mean and dividing by the standard deviation
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [11]:
# Save the number of predictors as number of columns
n_cols = predictors_norm.shape[1]
print(n_cols)

8


In [12]:
# Build a neural network

def regression_model():

    # build model
    model = Sequential() 

    # build layers: two hidden layers with 50 units
    model.add(Dense(50, activation='relu', input_shape=(n_cols, )))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # define an optimizer as minimization algorithm for training and loss meause
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [13]:
# Test and train the network

model = regression_model()

# fit the model: train and test at the same time, leave out 30% of the data for validation and train for 100 epochs
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100


/home/ap/miniconda3/envs/regression_keras/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 36ms/step - loss: 1633.3783 - val_loss: 1130.1722
Epoch 2/100
23/23 - 0s - 4ms/step - loss: 1491.5607 - val_loss: 998.1894
Epoch 3/100
23/23 - 0s - 4ms/step - loss: 1263.1042 - val_loss: 797.1871
Epoch 4/100
23/23 - 0s - 4ms/step - loss: 932.7864 - val_loss: 548.3672
Epoch 5/100
23/23 - 0s - 3ms/step - loss: 574.2189 - val_loss: 334.7046
Epoch 6/100
23/23 - 0s - 3ms/step - loss: 329.9432 - val_loss: 230.9771
Epoch 7/100
23/23 - 0s - 4ms/step - loss: 251.0463 - val_loss: 203.4082
Epoch 8/100
23/23 - 0s - 4ms/step - loss: 228.4900 - val_loss: 192.5143
Epoch 9/100
23/23 - 0s - 3ms/step - loss: 213.8058 - val_loss: 186.6275
Epoch 10/100
23/23 - 0s - 3ms/step - loss: 202.8533 - val_loss: 179.3984
Epoch 11/100
23/23 - 0s - 3ms/step - loss: 195.5338 - val_loss: 177.5751
Epoch 12/100
23/23 - 0s - 3ms/step - loss: 187.7848 - val_loss: 172.9742
Epoch 13/100
23/23 - 0s - 3ms/step - loss: 182.1400 - val_loss: 171.2306
Epoch 14/100
23/23 - 0s - 3ms/step - loss: 176.5279 - val_loss: 166